# 1 Understanding MDP's

## 1.1 Chess
State space: 8x8 spaces including integers to indicate the pieces / pawns on the board

action space: 8x8xN<br>
King, Queen, Bishop/Rook: Linear movements in 8 directions * max. 7 steps per direction. <br>
Knight movement: 8 possibilities. <br>
Pawn movements: 3 * Pawn promotions: 3<br>
N = 73 <br>
Reward: big reward for winning the game (negative for loosing) and small rewards for being in a material advantage
Policy: mapping the state space to the action space -> pi(8x8x73,8x8)

## 1.2 LunarLander
State space: S = { coordinate x, coordinate y, linear velocity in x, linear velocity in y, angular velocity, leg one, leg two} <br>
S = {[-90, 90], [-90, 90], [-5, 5], [-5, 5], [-3.1415, 3.1415], [0,1], [0,1]}<br>
Action space: 4 (do nothing, fire left, fire rigfht, fire main)<br>
Policy/reward: Land the spaceship on the landing pad.
in summary: reward is increased when the lander lands correctly without making too many movements.

## 1.3 Model Based RL
the reward function maps an observation and an action to a reward value that tells the agent how well it is doing.
E.g. in the CarRacing environment the agent gets a positive reward if it reaches a new tile and a small negative reward for every frame.
In Pong the agent gets a positive reward for scoring a point and a negative reward for losing a point.

the state transition function defines the next state given a current state and action. It can be deterministic or stochastic.
A deterministic example would be if in the GridWorld given that the action is going down the agent will end up one square below the original state
An example for a stochastic function would be if in the GridWorld the agent chooses an action but the action is only applied with a probability of 0.8 and it will perform a random action with a probability of 0.2

Discuss: No, they are not generally known. The agent has to learn these functions by taking actions and observing the states and rewards.

# 2 Implementing a GridWorld

## 2.1 Links:
1. https://towardsdatascience.com/reinforcement-learning-implement-grid-world-from-scratch-c5963765ebff
2. https://medium.com/mlearning-ai/applying-reinforcement-learning-algorithms-to-solve-gridworld-problems-29998406dd75
3. https://notebook.community/spro/practical-pytorch/reinforce-gridworld/reinforce-gridworld

In [4]:
import numpy as np

# inspired by: https://github.com/MJeremy2017/reinforcement-learning-implementation/blob/master/GridWorld/gridWorld.py
BOARD_ROWS = 3
BOARD_COLS = 5
# states = 3x5
WIN_STATE = (0, 2)
LOSE_STATE_I = (1, 0)
LOSE_STATE_II = (1, 4)
START = (2, 2)
DETERMINISTIC = True

class GridWorld(): 
    def __init__(self, state=START):
        # Initialize GridWorld board
        self.board = np.zeros([BOARD_ROWS, BOARD_COLS])

        self.state = state
        self.det = DETERMINISTIC
        
    def giveReward(self):
        if self.state == WIN_STATE:
            return 1
        elif (self.state == LOSE_STATE_I) or (self.state == LOSE_STATE_II):
            return -1
        else:
            return 0

    def nextPosition(self, action):
        """
        action: up, down, left, right
        return: next position
        """
        if self.det:
            if action == "up":
                nextState = (self.state[0] - 1, self.state[1])
            elif action == "down":
                nextState = (self.state[0] + 1, self.state[1])
            elif action == "left":
                nextState = (self.state[0], self.state[1] - 1)
            else: 
                nextState = (self.state[0], self.state[1] + 1)
            # if next state legal aka. if field is free
            if (nextState[0] >= 0) and (nextState[0] <= (BOARD_ROWS -1)):  
                if (nextState[1] >= 0) and (nextState[1] <= (BOARD_COLS -1)):
                    if nextState != (1,2):
                        return nextState
            return self.state

    def showBoard(self):
        """Show the GridWorld playfield in ASCII art"""
        self.board[self.state] = 1
        # Wall / barrier
        self.board[1,2] = -1
        # Win / Lose states
        self.board[WIN_STATE] = 2
        self.board[LOSE_STATE_I] = -2
        self.board[LOSE_STATE_II] = -2
        for i in range(0, BOARD_ROWS):
            print('-----------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                # Player
                if self.board[i, j] == 1:
                    token = '*'
                # Wall / barrier
                if self.board[i, j] == -1:
                    token = 'z'
                # Free fields
                if self.board[i, j] == 0:
                    token = '0'
                # Win state
                if self.board[i, j] == 2:
                    token = 'W'
                # Lose state
                if self.board[i, j] == -2:
                    token = 'L'
                out += token + ' | '
            print(out)
        print('-----------------')

GW = GridWorld()
GW.showBoard()

-----------------
| 0 | 0 | W | 0 | 0 | 
-----------------
| L | 0 | z | 0 | L | 
-----------------
| 0 | 0 | * | 0 | 0 | 
-----------------


In [5]:
class Agent():
    def __init__(self):
        self.GW = GridWorld()
        self.end = False
        # Set of actions
        self.action_list = ["up", "down", "left", "right"]

        # Add a dictionary to safe all state values for visualization later
        self.state_values = {}
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                self.state_values[(i, j)] = [0, 0]

    def distanceToWin(self):
        ydistance = self.GW.state[0] - WIN_STATE[0]
        xdistance = self.GW.state[1] - WIN_STATE[1]
        return xdistance, ydistance
    
    def chooseAct(self):
        xd, yd = self.distanceToWin()
        
        if xd==0 and yd>0:
            action = self.action_list[0]
        elif xd==0 and yd<0:
            action = self.action_list[1]
        elif xd>0 and yd==0:
            action = self.action_list[2]
        elif xd<0 and yd==0:
            action = self.action_list[3]
        # If x and y > 0 choose the smaller distance
        elif xd>yd:
            if xd>0:
                action = self.action_list[2]
            if xd<0:
                action = self.action_list[3]
        else:
            if yd>0:
                action = self.action_list[0]
            if yd<0:
                action = self.action_list[1]
        return action
    
    # Just for showcase purposes of one episode with print commands
    def ExamplemakeAct(self):
        if np.random.rand() > 0.3:
            action = self.chooseAct()
        else:
            action = self.action_list[np.random.randint(0,4)]
        print("Current position {} action {}".format(self.GW.state, action))
        self.GW.state = self.GW.nextPosition(action)
        self.reward = self.GW.giveReward()
        print("Next state", self.GW.state)
        print("---------------------")
        if self.reward==1 or self.reward==-1:
            # Reset game/ end episode
            self.end = True
            print("Game end, reward", self.reward)
            print("---------------------")

    def makeAct(self):
        # Choose action according to distance to goal in 70% of the time,
        # otherwise choose action random
        if np.random.rand() > 0.3:
            action = self.chooseAct()
        else:
            action = self.action_list[np.random.randint(0,4)]
        self.GW.state = self.GW.nextPosition(action)
        self.reward = self.GW.giveReward()
        if self.reward==1 or self.reward==-1:
            # End game
            self.end = True
        self.state_values[self.GW.state]
    
    def reset(self):
        self.GW = GridWorld()
        self.end = False

    def play(self):
        i = 0
        # Create empty list to safe states where the agent was in this episode
        self.state_list = []
        self.state_list.append(self.GW.state)
        # Create empty dictionary to safe MC-estimates
        self.erg = {}

        while self.end==False:
            self.makeAct()
            self.state_list.append(self.GW.state)
            i += 1

        # Remove duplicates from list of states
        self.state_list = list(dict.fromkeys(self.state_list))

        for s in reversed(self.state_list):
            # Add reward and state to dictionary of state values
            self.state_values[s][0] += self.reward
            self.state_values[s][1] += 1
        
        for s in ag.state_values:
            try:
                # Safe MC-estimates per k-many visits in state to result dictionary
                self.erg[s] = round(self.state_values[s][0]/self.state_values[s][1],3)
            except:
                # Prevent division by 0
                self.erg[s] = 0

        # Useful for Bugfixing
        #print('Game end, reward', self.reward)
        #print('Turns:',i)
        #print("---------------------")
        self.reset()

    def showValues(self):
        """Show MC-estimates per state aka. field in GridWorld"""
        for i in range(0, BOARD_ROWS):
            print('----------------------------------------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                out += str(self.erg[(i, j)]).ljust(6) + ' | '
            print(out)
        print('----------------------------------------------')
        
ag = Agent()
ag.ExamplemakeAct()
ag.play()

Current position (2, 2) action up
Next state (2, 2)
---------------------


In [6]:
for k in range(1,10001):
    ag.play()
    # Show MC-estimates per 50, 200, 500, 1000, 10000 
    if k in [50,200,500,1000,10000]:
        print("k =",k)
        print(ag.showValues())

k = 50
----------------------------------------------
| 1.0    | 1.0    | 1.0    | 1.0    | 1.0    | 
----------------------------------------------
| -1.0   | 1.0    | 0      | 0.565  | -1.0   | 
----------------------------------------------
| -1.0   | 0.793  | 0.608  | 0.385  | -1.0   | 
----------------------------------------------
None
k = 200
----------------------------------------------
| 1.0    | 1.0    | 1.0    | 1.0    | 1.0    | 
----------------------------------------------
| -1.0   | 0.814  | 0      | 0.744  | -1.0   | 
----------------------------------------------
| -1.0   | 0.661  | 0.602  | 0.535  | -1.0   | 
----------------------------------------------
None
k = 500
----------------------------------------------
| 0.882  | 0.991  | 1.0    | 0.979  | 0.857  | 
----------------------------------------------
| -1.0   | 0.798  | 0      | 0.771  | -1.0   | 
----------------------------------------------
| -0.769 | 0.632  | 0.601  | 0.561  | -0.879 | 
------------------